In [ ]:
pip install imutils

In [ ]:
# import the necessary packages
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from pandas import DataFrame
import matplotlib.pyplot as plt
import ast 
import numpy as np
import pandas as pd
import cv2
import os
import random
from imutils import paths
from sklearn.preprocessing import LabelBinarizer

burada klasörden görüntülerin isimlerini liste halinde çekiyoruz

In [ ]:
img_name = list(map(lambda x: x, os.listdir('../input/illinois-doc-labeled-faces-dataset/front/front/')))

çektiğimiz isimlerle dataframe oluşturuyoruz

In [ ]:
df = pd.DataFrame(data = img_name, index =None, columns = ['filename'])

dataframe de yeni kolon açarak görüntülere suçlu etiketi ekleniyor.

In [ ]:
df.insert(1,"criminal","suclu")

In [ ]:
df

In [ ]:
data = []
labels = []

burada klasörden görüntülerin yollarını çekiyoruz daha sonra bu yollar resimleri klasörden okumamıza yarayacak

In [ ]:
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images('../input/illinois-doc-labeled-faces-dataset/front/front/')))

çekilen dosyanın ne kadar veri içerdiğine baklıyor

In [ ]:
len(imagePaths)

In [ ]:
imagePaths[0:5]

In [ ]:
IMAGE_DIMS = (224, 224, 3)

In [ ]:
import random

burada görüntüler çekilerek ileride eğitebilmek için özelik ve giriş değerleri belirleniyor.

In [ ]:
# loop over the input images
for i in random.sample(range(len(imagePaths)),5000):
	# load the image, pre-process it, and store it in the data list
    img=imagePaths[i]
    image = cv2.imread(f"{img}")
    image1 = cv2.resize(image,(224,224))
    image1 = img_to_array(image1)
    data.append(image1)
    

	# extract the class label from the image path and update the
	# labels list
	# sınıf etiketini görüntü yolundan çıkarın ve etiket listesini güncelleyin
    label = "suclu"
    labels.append(label)

In [ ]:
len(data)

In [ ]:
labels[0:8]

In [ ]:
# scale the raw pixel intensities to the range [0, 1]
# ham piksel yoğunluklarını [0, 1] aralığına ölçeklendirin
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [ ]:
# binarize the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
len(lb.classes_)

In [ ]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
#% 80'i kullanarak verileri eğitim ve test bölümlerine ayırın
# eğitim verileri ve kalan% 20 test için
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.1, random_state=42)

In [ ]:
# construct the image generator for data augmentation
# Veri büyütme için görüntü oluşturucuyu oluşturun
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam
from tensorflow.keras.regularizers import l1, l2, L1L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
# load the VGG16 network, ensuring the head FC layers are left off
vgg = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
softmaxHead = Dense(512, activation="relu")(flatten)
softmaxHead = Dropout(0.5)(softmaxHead)
softmaxHead = Dense(512, activation="relu")(softmaxHead)
softmaxHead = Dropout(0.3)(softmaxHead)
softmaxHead = Dense(len(lb.classes_), activation="sigmoid")(softmaxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=softmaxHead)
print(model.summary())

In [ ]:
EPOCHS = 20
INIT_LR =1e-3
BS = 32

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="mse", optimizer=opt,metrics=["accuracy"])
# train the network for bounding box regression
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
print("[INFO] training bounding box regressor...")
H = model.fit(
	x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1, callbacks = [es])

In [ ]:
BASE_OUTPUT = '../working/'
MODEL_PATH = os.path.sep.join([BASE_OUTPUT, "detector1.h5"])
# serialize the model to disk
print("[INFO] saving object detector model...")
model.save(MODEL_PATH, save_format="h5")

In [ ]:
score = model.evaluate(x=testX,y=testY,batch_size=BS)
score
print('Score Accuracy : {:.2f}%'.format(score[1]*100))

In [ ]:
pred_Y = model.predict(testX, batch_size = BS, verbose = True)
pred_Y_cat = np.argmax(pred_Y, -1)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

confusion_matrix=(confusion_matrix(np.argmax(testY,-1), pred_Y_cat))

In [ ]:
lb.classes_[:]

In [ ]:
import seaborn as sns
LABELS=['suclu']
plt.figure(figsize=(10,8))
sns.heatmap(confusion_matrix, xticklabels=LABELS, yticklabels=LABELS, annot=True, fmt = 'd')
plt.title('Confusion Matrix')
plt.ylabel('True class')
plt.xlabel('Predicted class')
plt.show()

In [ ]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import mimetypes
import argparse
import imutils
import cv2
import os

In [ ]:
print("[INFO] loading object detector...")
model = load_model('./detector1.h5')

In [ ]:
def predict(imagePath):
    # load the input image (in Keras format) from disk and preprocess
    # it, scaling the pixel intensities to the range [0, 1]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    output = image.copy()
    image = cv2.resize(image, (224, 224))
    image = image.astype("float") / 255.0
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    # make bounding box predictions on the input image
    proba = model.predict(image)[0]
    idx = np.argmax(proba)
    label = lb.classes_[idx]
    label = "{} olma ihtimali: {:.2f}% ".format(label, proba[idx] * 100)
    cv2.putText(output, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,
        0.7, (255, 0, 0), 2)
        
    # show the output image
    fig=plt.figure(figsize=(10,10))
    plt.grid(b=None)
    plt.axis('off')
    return plt.imshow(output)

In [ ]:
imagePath = "../input/illinois-doc-labeled-faces-dataset/front/front/A00147.jpg"
predict(imagePath)

In [ ]:
imagePath = "../input/illinois-doc-labeled-faces-dataset/front/front/A01181.jpg"
predict(imagePath)

In [ ]:
imagePath = "../input/illinois-doc-labeled-faces-dataset/front/front/A01759.jpg"
predict(imagePath)